In [1]:
%matplotlib inline
import numpy as np
import glob
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
import os

In [2]:
list_ = []
for name in os.listdir('500GeV_res/csv/'):
    filename = '500GeV_res/csv/' + name
    df = pd.read_csv(filename,delimiter= ' ')
    list_.append(df)

In [3]:
full_dataframe = pd.concat(list_,ignore_index=True)

In [4]:
full_dataframe.shape

(608979, 20)

In [5]:
y_tcHW = full_dataframe['tcHW']
y_cHW = full_dataframe['cHW']
xsec = full_dataframe['xsec']

In [6]:
y_target = np.array([y_cHW, y_tcHW, xsec]).T

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor

In [8]:
full_dataframe.drop(labels=['cHW','tcHW','xsec'],axis=1,inplace=True)

In [9]:
X = full_dataframe

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y_target,
                                                    train_size=0.8,
                                                    random_state=4)

/public/home/fp4303/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 12, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [13]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [10, 14, 18, 23, 27, 32, 36, 41, 45, 50], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [14]:
regr_rf = RandomForestRegressor()

In [15]:
print(regr_rf.get_params())

{'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': 1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [16]:
rf_random = RandomizedSearchCV(estimator = regr_rf, param_distributions = random_grid,
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [17]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=14, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=4, bootstrap=True 
[CV] n_estimators=14, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=4, bootstrap=True 
[CV] n_estimators=14, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=4, bootstrap=True 
[CV] n_estimators=50, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=2, bootstrap=True 
[CV] n_estimators=50, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=2, bootstrap=True 
[CV] n_estimators=50, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=2, bootstrap=True 
[CV] n_estimators=32, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=2, bootstrap=False 
[CV] n_estimators=32, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=2, bootstrap=False 
[CV] n_estimators=32, min_samples_split=5, min_

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   27.4s


[CV]  n_estimators=32, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=2, bootstrap=False, total=  26.8s
[CV] n_estimators=27, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=6, bootstrap=False 
[CV]  n_estimators=50, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=2, bootstrap=True, total=  29.1s
[CV] n_estimators=27, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=6, bootstrap=False 
[CV]  n_estimators=50, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=2, bootstrap=True, total=  29.8s
[CV] n_estimators=45, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=2, bootstrap=False 
[CV]  n_estimators=23, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=4, bootstrap=False, total=  38.8s
[CV] n_estimators=45, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=2, bootstrap=False 
[CV]  n_estimators=23, min_samples_split=5, mi

[CV] n_estimators=18, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=9, bootstrap=True 
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=4, bootstrap=False, total= 4.5min
[CV] n_estimators=45, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=4, bootstrap=False 
[CV]  n_estimators=27, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=6, bootstrap=False, total= 4.2min
[CV] n_estimators=45, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=4, bootstrap=False 
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=4, bootstrap=False, total= 4.8min
[CV] n_estimators=45, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=4, bootstrap=False 
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=4, bootstrap=False, total= 4.9min
[CV] n_estimators=41, min_samples_split=5,

[CV]  n_estimators=32, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=3, bootstrap=False, total=  35.0s
[CV] n_estimators=23, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=6, bootstrap=False 
[CV]  n_estimators=32, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=3, bootstrap=False, total=  32.5s
[CV] n_estimators=23, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=6, bootstrap=False 
[CV]  n_estimators=32, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=3, bootstrap=False, total=  33.4s
[CV] n_estimators=23, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=11, bootstrap=False 
[CV]  n_estimators=36, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=5, bootstrap=False, total= 4.1min
[CV] n_estimators=23, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=11, bootstrap=False 
[CV]  n_estimators=23, min_samples_spli

...

[CV]  n_estimators=23, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=3, bootstrap=False, total=  23.0s
[CV] n_estimators=23, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=3, bootstrap=False 
[CV]  n_estimators=23, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=3, bootstrap=False, total=  22.5s
[CV] n_estimators=45, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=7, bootstrap=True 
[CV]  n_estimators=36, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=9, bootstrap=False, total= 1.8min
[CV] n_estimators=45, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=7, bootstrap=True 
[CV]  n_estimators=36, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=9, bootstrap=False, total= 1.9min
[CV] n_estimators=45, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=7, bootstrap=True 
[CV]  n_estimators=36, min_samples_split=2, min

[CV] n_estimators=23, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=18, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=11, bootstrap=True, total= 3.3min
[CV] n_estimators=23, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=18, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=11, bootstrap=True, total= 2.8min
[CV] n_estimators=23, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=18, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=11, bootstrap=True, total= 3.1min
[CV] n_estimators=23, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=8, bootstrap=False 
[CV]  n_estimators=23, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=8, bootstrap=False, total= 1.0min
[CV] n_estimators=23, min_sam

[CV]  n_estimators=41, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=21.9min
[CV] n_estimators=18, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=4, bootstrap=False 
[CV]  n_estimators=41, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=22.1min
[CV] n_estimators=14, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=5, bootstrap=False 
[CV]  n_estimators=18, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=4, bootstrap=False, total=  25.4s
[CV] n_estimators=14, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=5, bootstrap=False 
[CV]  n_estimators=18, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=4, bootstrap=False, total=  25.6s
[CV] n_estimators=14, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=5, bootstrap=False 
[CV]  n_estimators=18, min_samples_

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 75.6min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [10, 14, 18, 23, 27, 32, 36, 41, 45, 50], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [18]:
print(rf_random.best_params_)

{'n_estimators': 27, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 2, 'bootstrap': True}


In [19]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100*np.mean(errors/test_labels)
    accuracy = 100 - mape
    print('Model performance')
    print('Average error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%'.format(accuracy))
    return accuracy

In [20]:
base_model = RandomForestRegressor(n_estimators=10,random_state=42)
base_model.fit(X_train,y_train)
base_accuracy = evaluate(base_model, X_test,y_test)

Model performance
Average error: 0.0521 degrees.
Accuracy = nan%


/public/home/fp4303/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/public/home/fp4303/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [21]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

Model performance
Average error: 0.0495 degrees.
Accuracy = -inf%


/public/home/fp4303/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
